In [6]:
import cv2
from PIL import Image
import numpy as np
import os

# Load the original image
image_path = r"C:\Users\sanja\OneDrive\Pictures\img (2)\img\Canadian-Passport-Photos020.webp"
image = cv2.imread(image_path)

# Define the directory to save augmented images
output_dir = "augmented_images"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define the number of augmented images you want to generate
num_augmented_images = 100

# Perform image augmentation
for i in range(num_augmented_images):
    # Apply random transformations to the image
    augmented_image = image.copy()
    
    # Example transformations (you can customize these as needed)
    # Rotate the image by a random angle between -10 and 10 degrees
    angle = np.random.randint(-10, 10)
    augmented_image = Image.fromarray(augmented_image)
    augmented_image = augmented_image.rotate(angle, resample=Image.BICUBIC, fillcolor=(255, 255, 255))
    augmented_image = np.array(augmented_image)
    
    # Save the augmented image
    output_path = os.path.join(output_dir, f"augmented_image_{i}.jpg")
    cv2.imwrite(output_path, augmented_image)

print("Image augmentation completed.")

Image augmentation completed.


In [3]:
# load dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [3]:
# load and create labeled data
data_dir = r"D:\Sam2\Capston2\Capston002\Capstone_Final_project\Images"
batch_size = 32
epochs = 20
input_shape = (224, 224, 3)
image_paths = []
labels = []

for category in os.listdir(data_dir):
    category_dir = os.path.join(data_dir, category)
    if os.path.isdir(category_dir):
        for image_filename in os.listdir(category_dir):
            if image_filename.endswith('.jpg'):
                image_path = os.path.join(category_dir, image_filename)
                image_paths.append(image_path)
                labels.append(category)

In [4]:
# split dataset
train_image_paths, test_image_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42)

In [5]:
# create data generator for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_dataframe(
    pd.DataFrame({'image_path': train_image_paths, 'label': train_labels}),
    x_col='image_path',
    y_col='label',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

Found 800 validated image filenames belonging to 10 classes.


In [6]:
# build model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax')) 

D:\Sam2\Capston2\Capston002\Capstone_Final_project\cap_venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [7]:
from tensorflow.keras.optimizers import Adam

# compile 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0001),
              metrics=['accuracy'])

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the filepath with '.keras' extension
checkpoint = ModelCheckpoint('Face-Recognition-based-Attendance-System_model.h5.keras',
                             monitor='val_loss',
                             save_best_only=True,
                             verbose=1)

In [9]:
# train model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[checkpoint]
)

Epoch 1/20


D:\Sam2\Capston2\Capston002\Capstone_Final_project\cap_venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.1367 - loss: 2.3456
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/20


D:\Sam2\Capston2\Capston002\Capstone_Final_project\cap_venv\Lib\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
C:\Users\sanja\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


25/25 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.3346 - loss: 1.9413
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 329us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 29s 964ms/step - accuracy: 0.4997 - loss: 1.4190
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 131us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 28s 933ms/step - accuracy: 0.6329 - loss: 1.0811
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 130us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.7391 - loss: 0.8223
Epoch 10/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 131us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 11/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 28s 954ms/step - accuracy: 0.7923 - loss: 0.5926
Epoch 12/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 115us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 13/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 28s 926ms/step - accuracy: 0.8865 - loss: 0.4554
Epoch 14/20
25/25 ━━━

In [10]:
# save model
model.save('Face-Recognition-based-Attendance-System_model.h5')

In [11]:
# evaluate on test data
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_dataframe(
    pd.DataFrame({'image_path': test_image_paths, 'label': test_labels}),
    x_col='image_path',
    y_col='label',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

loss, accuracy = model.evaluate(test_generator)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy * 100:.2f}%')

Found 200 validated image filenames belonging to 10 classes.


D:\Sam2\Capston2\Capston002\Capstone_Final_project\cap_venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 577ms/step - accuracy: 0.9702 - loss: 0.0779
Test loss: 0.0824
Test accuracy: 96.50%


In [12]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

In [13]:
# Load the pre-trained CNN model
model = load_model(r"D:\Sam2\Capston2\Capston002\Capstone_Final_project\Face-Recognition-based-Attendance-System_model.h5")

In [14]:
# Compile the model (if not compiled before loading)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [24]:
# Load the image you want to classify
img_path = r"D:\Sam2\Capston2\Capston002\Capstone_Final_project\Images\Kunal\augmented_image_1.jpg"  # Provide the path to your image
img = image.load_img(img_path, target_size=(224, 224))  # Assuming input shape of the model is (224, 224)

In [25]:
# Preprocess the image
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Normalize the pixel values

In [26]:
# Use the model to predict the class of the image
predictions = model.predict(img_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


In [27]:
# Map the predicted class index to the corresponding category label
class_labels = ['Jasan', 'Jaspreet', 'Karan', 'Kunal', 'Mohit', 'Preyas', 'Princeton', 'Sanjay', 'Shaid' , 'Vandana']
predicted_class_index = np.argmax(predictions)
predicted_class_label = class_labels[predicted_class_index]

print("Predicted class:", predicted_class_label)

Predicted class: Kunal
